In [ ]:
!pip install dimod
!pip install tqdm
!pip install dwave-neal
!pip install networkx
!pip install matplotlib
!pip install pandas

import numpy as np
import pandas as pd
import random
import json
import time
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.notebook import tqdm
import dimod
import neal
import os
import gc
from collections import defaultdict, Counter, deque
from google.colab import files

In [ ]:
# import os
# import glob

# restart
# patterns_to_delete = [
#     "benchmark_summary_*.csv",
#     "problem_instance_6p_*.json",
#     "trust_over_time_benchmark_*.csv"
# ]

# deleted_files = []

# for pattern in patterns_to_delete:
#     for file_path in glob.glob(pattern):
#         os.remove(file_path)
#         deleted_files.append(file_path)

# print("Deleted files:")
# for f in deleted_files:
#     print("Deleted", f)

# if not deleted_files:
#     print("No files matched.")

In [ ]:
TOTAL_ROUNDS = 100
BLOCK_FRACTION = 0.20
LOCAL_VIEW_RADIUS = 2
NUM_READS = 100
SEVERITY_LEVELS = [0.25, 0.5, 0.75, 0.9]
SPOOF_FRAC_RANGE = (0.3, 0.6)
SPOOF_SEVERITY = 0.25

In [ ]:
uploaded = files.upload()
benchmark_files = [str(f) for f in uploaded.keys()]

print(f"Uploaded {len(benchmark_files)} files:")
for i, fname in enumerate(benchmark_files, 1):
    print(f"  {i}. {fname}")

Saving problem_instance_6p_235308.json to problem_instance_6p_235308.json
Saving problem_instance_6p_235307.json to problem_instance_6p_235307.json
Saving problem_instance_6p_235306.json to problem_instance_6p_235306.json
Saving problem_instance_6p_235305.json to problem_instance_6p_235305.json
Saving problem_instance_6p_235304.json to problem_instance_6p_235304.json
Saving problem_instance_6p_235303.json to problem_instance_6p_235303.json
Saving problem_instance_6p_235302.json to problem_instance_6p_235302.json
Saving problem_instance_6p_235301.json to problem_instance_6p_235301.json
Saving problem_instance_6p_235300.json to problem_instance_6p_235300.json
Saving problem_instance_6p_235299.json to problem_instance_6p_235299.json
Saving problem_instance_6p_235294.json to problem_instance_6p_235294.json
Saving problem_instance_6p_235285.json to problem_instance_6p_235285.json
Saving problem_instance_6p_235282.json to problem_instance_6p_235282.json
Saving problem_instance_6p_235281.json

In [ ]:
#graph helpers

def build_benchmark_graph(data):
    G = nx.Graph()
    num_tasks = len(data["T_e"])

    for i in range(num_tasks):
        urgency = data["T_e"][i] / 100.0
        x, y = data["task_locations"][i]
        trust = round(random.uniform(0.5, 0.95), 2)
        G.add_node(f"T{i}", type="task", urgency=urgency, x=x, y=y, structural_integrity=1.0, trust=trust)

    for i in range(num_tasks):
        for j in range(i + 1, num_tasks):
            dist = data["T_t"][i][j]
            G.add_edge(f"T{i}", f"T{j}", distance=dist, status="open")

    return G, [f"T{i}" for i in range(num_tasks)]

def assign_chained_precedence(graph, chain_length=3, num_chains=3):
    tasks = [n for n in graph.nodes if graph.nodes[n].get('type') == 'task']
    chains = []
    used = set()

    for _ in range(num_chains):
        chain = random.sample([t for t in tasks if t not in used], chain_length)
        for i in range(len(chain) - 1):
            graph.nodes[chain[i + 1]]['must_follow'] = chain[i]
            chains.append((chain[i], chain[i + 1]))
            used.add(chain[i])
    return chains

def define_precedence_map(precedence_pairs):
    mapping = {}
    for a, b in precedence_pairs:
        mapping.setdefault(b, []).append(a)
    return mapping

def simulate_outcome_with_path_check(graph, robot, task, exit_rooms):
    temp_graph = graph.copy()
    blocked_nodes = [n for n, d in graph.nodes(data=True) if d.get("blocked")]
    temp_graph.remove_nodes_from(blocked_nodes)

    start = robot["assigned_room"]
    end_task = task

    try:
        path_to_task = nx.shortest_path(temp_graph, source=start, target=end_task)
    except nx.NetworkXNoPath:
        return False

    can_exit = False
    for exit_room in exit_rooms:
        try:
            path_to_exit = nx.shortest_path(temp_graph, source=end_task, target=exit_room)
            can_exit = True
            break
        except nx.NetworkXNoPath:
            continue

    return can_exit

In [ ]:
#robot and trust helpers

def init_benchmark_robots(data):
    robots = {}
    skill_vectors = data["R"]
    for i, skills in enumerate(skill_vectors):
        trust = round(random.uniform(0.5, 0.95), 2)
        robots[i] = {
            'trust': trust,
            'trust_history': [],
            'compromised': False,
            'success_count': 0,
            'fail_count': 0,
            'assigned_room': f"T{random.randint(0, len(data['T_e']) - 1)}",
            'spoofed_room': f"T{random.randint(0, len(data['T_e']) - 1)}",
            'skills': skills,
            'role': 'worker',
            'initial_trust': trust,
        }
    return robots

def switch_robot_roles(robots, round_num, switch_round=50):
    if round_num == switch_round:
        for r in robots:
            r['skills'] = np.random.permutation(r['skills'])

def reset_spoofed_rooms(robots):
    for r in robots.values():
        r['spoofed_room'] = r['assigned_room']

def apply_environmental_noise(robots, noise_level=0.05):
    for r in robots.values():
        if not r['compromised']:
            noise = random.uniform(1 - noise_level, 1 + noise_level)
            r['trust'] = min(max(r['trust'] * noise, 0), 1)

def add_trust_noise(true_trust, noise_level=0.1):
    noisy = true_trust + np.random.normal(0, noise_level, size=true_trust.shape)
    return np.clip(noisy, 0, 1)

def cure_spoofed_robots(robots, budget_frac=0.3, recovery_boost=0.5):
    num_to_cure = int(len(robots) * budget_frac)
    compromised_agents = [r for r in robots.values() if r['compromised']]
    agents_to_cure = random.sample(compromised_agents, min(num_to_cure, len(compromised_agents)))
    for r in agents_to_cure:
        r['compromised'] = False
        r['trust'] = min(r['trust'] + recovery_boost, 1.0)

def spoofing_attack(robots, tasks, dist_matrix, spoof_frac_range=(0.05, 0.2), severity=0.15):
    num_agents = len(robots)
    spoof_frac = random.uniform(*spoof_frac_range)
    num_spoofed = int(num_agents * spoof_frac)
    candidates = sorted(robots.items(), key=lambda x: -x[1]['trust'])
    spoof_targets = [r for r_id, r in candidates[:num_spoofed]]

    for robot in spoof_targets:
        robot['compromised'] = True
        if robot['assigned_room'] and len(tasks) > 0:
            alt = random.choice(list(tasks))
            robot['spoofed_room'] = alt
        else:
            robot['spoofed_room'] = robot['assigned_room']

def spoofing_attack_target_high_trust(robots, tasks, dist_matrix, spoof_frac_range=(0.05, 0.2), severity=0.1):
    num_agents = len(robots)
    spoof_frac = random.uniform(*spoof_frac_range)
    num_to_spoof = int(spoof_frac * num_agents)
    candidates = sorted(robots.items(), key=lambda x: x[1]["trust"], reverse=True)
    selected = random.sample(candidates[:num_agents//2], min(num_to_spoof, len(candidates)))

    for rid, robot in selected:
        robot["compromised"] = True
        robot["spoofed_room"] = random.choice(tasks)
        robot["trust"] = max(0.1, robot["trust"] - severity)

def inject_fake_precedence(robots, task_pool):
    for r in robots.values():
        fake_pairs = random.sample(task_pool, 2)
        r['perceived_precedence'] = [(fake_pairs[0], fake_pairs[1])]


def simulate_comms_failures(robots, drop_prob=0.2):
    for r in robots.values():
        if random.random() < drop_prob:
            r['comms_down'] = True
        else:
            r['comms_down'] = False

def apply_local_view(task_list, agent_pos, graph, radius=2):
    local_tasks = []
    lengths = nx.single_source_shortest_path_length(graph, agent_pos, cutoff=radius)
    for task in task_list:
        if task in lengths:
            local_tasks.append(task)
    return local_tasks

In [ ]:
#task management

def decay_task_urgency(graph, decay_rate=0.2):
    for t in graph.nodes:
        if graph.nodes[t].get('type') == 'task':
            urgency = graph.nodes[t].get('urgency', 0.2)
            graph.nodes[t]['urgency'] = max(urgency - decay_rate, 0)

def update_urgency(graph, robots, tasks):
    treated = set(r['spoofed_room'] for r in robots.values())
    nearby_counts = Counter(treated)

    for task in tasks:
        curr = graph.nodes[task]['urgency']
        boost = 0.02 + 0.01 * nearby_counts.get(task, 0)
        if task in treated:
            graph.nodes[task]['urgency'] = round(max(0.1, curr - 0.05), 2)
        else:
            graph.nodes[task]['urgency'] = round(min(1.0, curr + boost), 2)


def update_dynamic_blockages(graph, block_prob=0.03):
    for u, v in list(graph.edges()):
        if random.random() < block_prob:
            graph.edges[u, v]['status'] = 'blocked'

def count_blocked_edges(graph):
    return sum(1 for _, _, d in graph.edges(data=True) if d.get('status') == 'blocked')

def assign_deadlines(graph, max_deadline=25):
    for node in graph.nodes:
        if graph.nodes[node].get("type") == "task":
            graph.nodes[node]['deadline'] = random.randint(10, max_deadline)

def check_expired_tasks(graph, current_round):
    expired = []
    for node in graph.nodes:
        if graph.nodes[node].get("type") == "task" and 'deadline' in graph.nodes[node]:
            if current_round > graph.nodes[node]['deadline']:
                expired.append(node)
    return expired

In [ ]:
#constraints

def assign_exit_rooms(graph, exit_count=3):
    task_nodes = [n for n in graph.nodes if graph.nodes[n].get('type') == 'task']
    exit_rooms = random.sample(task_nodes, min(exit_count, len(task_nodes)))
    for room in exit_rooms:
        graph.nodes[room]['is_exit'] = True
    return exit_rooms


def count_exit_violations(assignments, graph, exit_rooms):
    count = 0
    for a, room in assignments.items():
        if isinstance(room, list):
            room = room[0] if room else None
        if room in exit_rooms:
            count += 1
    return count


In [ ]:
#distance and cost

def get_distance_matrix(data):
    dist_matrix = {}
    num_tasks = len(data["T_e"])
    for i in range(num_tasks):
        for j in range(num_tasks):
            dist_matrix[(f"T{i}", f"T{j}")] = data["T_t"][i][j]
    return dist_matrix

def get_visible_tasks(robot, graph, radius=2):
    visible = []
    for task in graph.nodes:
        if graph.nodes[task].get("type") == "task":
            try:
                path_len = nx.shortest_path_length(graph, source=robot['position'], target=task)
                if path_len <= radius:
                    visible.append(task)
            except nx.NetworkXNoPath:
                continue
    return visible

def precompute_static_costs(robots, tasks, graph):
    static_terms = {}
    quad = {}
    agent_ids = list(robots.keys())

    for agent_id, robot in robots.items():
        for t_idx, task in enumerate(tasks):
            data = graph.nodes[task]
            skill = robot['skills'][t_idx % len(robot['skills'])]
            base = data.get('structural_integrity', 1.0) * skill
            static_terms[(agent_id, task)] = base - data.get('defense_level', 0.0)

    for task in tasks:
        occ = graph.nodes[task].get('occupants', 1)
        synergy = -0.5 * occ if occ >= 2 else 0
        for i in range(len(agent_ids)):
            for j in range(i + 1, len(agent_ids)):
                pair = ((agent_ids[i], task), (agent_ids[j], task))
                quad[pair] = 10.0 + synergy

    return static_terms, quad

def update_dynamic_costs_np(robots, tasks, graph, static_terms, dist_matrix,
                            dist_penalty=0.05, comp_penalty=0.5,
                            local_radius=LOCAL_VIEW_RADIUS, duplicate_soft_penalty=3.0):
    cost_dict = {}
    task_load = Counter()

    for agent_id, r in robots.items():
        for t in tasks:
            dist = dist_matrix.get((r['assigned_room'], t), 999)
            if dist > local_radius and graph.nodes[t].get('urgency', 0.3) < 0.7:
                continue

            key = (agent_id, t)
            base = static_terms.get(key, 1.0)
            trust = r['trust']
            urgency = graph.nodes[t].get('urgency', 0.2)
            gain_factor = 8.0 if trust >= 0.75 else 5.0
            urgency_adj = urgency ** 2 if urgency >= 0.4 else urgency
            cost = - urgency_adj * trust * base * gain_factor + dist_penalty * dist


            if r['compromised']:
                cost += comp_penalty
            if trust < 0.4 and urgency < 0.5:
                cost += 0.3

            cost_dict[key] = cost
            task_load[t] += 1

    for (agent_id, task), val in cost_dict.items():
        load = task_load[task]
        if load > 1:
            cost_dict[(agent_id, task)] += duplicate_soft_penalty * (load - 1)

    if cost_dict:
        min_cost = min(cost_dict.values())
        max_cost = max(cost_dict.values())
        range_cost = max_cost - min_cost + 1e-6
        for k in cost_dict:
            cost_dict[k] = (cost_dict[k] - min_cost) / range_cost

    return cost_dict


def hardened_greedy_assignment(robots, tasks, graph, precedence_map, dist_matrix, local_radius=2):
    task_to_agents = defaultdict(list)
    for robot_id, robot in robots.items():
        agent_pos = robot["assigned_room"]
        viewable_tasks = apply_local_view(tasks, agent_pos, graph, radius=local_radius)

        eligible_tasks = []
        for task in viewable_tasks:
            deps_met = all(any(r["assigned_room"] == dep for r in robots.values()) for dep in precedence_map.get(task, []))
            if deps_met:
                eligible_tasks.append(task)

        best_task = None
        best_score = float('-inf')
        for task in eligible_tasks:
            dist = dist_matrix.get((agent_pos, task), 50)
            urgency = graph.nodes[task].get("urgency", 0)
            score = urgency - 0.25 * dist
            if score > best_score:
                best_score = score
                best_task = task

        if best_task:
            robot["assigned_room"] = best_task
            task_to_agents[best_task].append(robot_id)

    return task_to_agents



def hardened_greedy_assignment(robots, tasks, graph, precedence_map, dist_matrix, local_radius=2):
    assignments = {}
    for robot_id, robot in robots.items():
        agent_pos = robot["assigned_room"]
        viewable_tasks = [t for t in tasks if dist_matrix.get((agent_pos, t), 999) <= local_radius]

        eligible_tasks = []
        for task in viewable_tasks:
            deps_met = all(any(r["assigned_room"] == dep for r in robots.values()) for dep in precedence_map.get(task, []))
            if deps_met:
                eligible_tasks.append(task)

        best_task = None
        best_score = float('-inf')
        for task in eligible_tasks:
            dist = dist_matrix.get((agent_pos, task), 50)
            urgency = graph.nodes[task].get("urgency", 0)
            score = urgency - 0.25 * dist
            if score > best_score:
                best_score = score
                best_task = task

        if best_task:
            robot["assigned_room"] = best_task
            assignments[robot_id] = best_task

    return assignments


In [ ]:
#solvers

def build_or_update_bqm(bqm, cost_dict, quad_penalties):
    if bqm is None:
        bqm = dimod.BinaryQuadraticModel('BINARY')
        for var in cost_dict:
            bqm.add_variable(var, 0.0)
        for (var1, var2), penalty in quad_penalties.items():
            bqm.add_interaction(var1, var2, penalty)
    for var, cost in cost_dict.items():
        bqm.set_linear(var, cost)
    return bqm

def apply_qubo_assignments(result, robots, tasks, precedence_map, graph):
    assignments = {}
    assigned_tasks = set()
    assigned_rooms = set(r['assigned_room'] for r in robots.values())
    for (agent_id, task), val in result.items():
        if val != 1 or task in assigned_tasks:
            continue
        deps = precedence_map.get(task, [])
        if all(dep in assigned_rooms for dep in deps):
            robots[agent_id]['assigned_room'] = task
            assignments[agent_id] = task
            assigned_tasks.add(task)
        else:
            continue
    return assignments


In [ ]:
def run_metrics(robots, assignments, tasks, precedence_map, dist_matrix, graph, exit_rooms):
    total_agents = len(robots)
    assignment_counts = Counter(tuple(v) if isinstance(v, list) else v for v in assignments.values())
    spoofed_agents = [r for r in robots.values() if r.get("compromised")]

    distances = []
    urgency_vals = []
    recovery_vals = []
    success_vals = []
    reward_vals = []
    trust_coord_vals = []
    team_assignments = defaultdict(list)

    duplicate_penalty = 0
    precedence_violations = 0
    bait_penalty = 0
    exit_violations = 0

    for agent_id, assigned_task in assignments.items():
        if agent_id not in robots:
            continue

        r = robots[agent_id]
        room = assigned_task[0] if isinstance(assigned_task, list) and assigned_task else assigned_task
        if not isinstance(room, str) or room not in graph.nodes:
            continue

        if not simulate_outcome_with_path_check(graph, r, room, exit_rooms):
            r["fail_count"] += 1
            continue

        if r.get("last_room") == room:
            r["fail_count"] += 1
            continue

        r["last_room"] = room

        if r.get("must_exit"):
            if room not in exit_rooms:
                exit_violations += 1
                r["fail_count"] += 1
                trust_coord_vals.append(0)
                continue
            else:
                r["must_exit"] = False
                reward_vals.append(r.get("pending_reward", 0))
                r["pending_reward"] = 0
                trust_coord_vals.append(r["trust"])
                r["success_count"] += 1
                r["trust"] = min(1.0, r["trust"] + 0.01)
        else:
            r["must_exit"] = True
            urgency = graph.nodes[room].get("urgency", 0)
            r["pending_reward"] = urgency * r["trust"]

        success_vals.append(1)
        urgency_vals.append(graph.nodes[room].get("urgency", 0))
        dist = dist_matrix.get((r["assigned_room"], room), 5.0)
        distances.append(dist)
        team_assignments[room].append(agent_id)

        if assignment_counts[room] > 1:
            duplicate_penalty += 1

        if room in precedence_map:
            deps = precedence_map[room]
            visited = [r["assigned_room"] for r in robots.values() if r["assigned_room"] in deps]
            if not all(dep in visited for dep in deps):
                precedence_violations += 1

        integrity = graph.nodes[room].get("integrity", 1.0)
        urgency = graph.nodes[room].get("urgency", 0.0)
        if urgency < 0.3 and integrity > 0.8:
            bait_penalty += 1

        total_attempts = r["success_count"] + r["fail_count"]
        recovery = r["success_count"] / total_attempts if total_attempts > 0 else 0
        recovery_vals.append(recovery)

    avg_trust = round(np.mean([r["trust"] for r in robots.values()]), 3)
    avg_urgency = round(np.mean(urgency_vals), 3) if urgency_vals else 0.0
    avg_recovery = round(np.mean(recovery_vals), 3) if recovery_vals else 0.0
    avg_coord_success = round(np.mean(success_vals), 3) if success_vals else 0.0
    avg_spoofed = round(len(spoofed_agents) / total_agents, 3)
    avg_distance = round(np.mean(distances), 2) if distances else 0.0
    avg_violations = round(precedence_violations / len(assignments), 3) if assignments else 0.0
    avg_duplicates = round(duplicate_penalty / len(assignments), 3) if assignments else 0.0
    coalition_success = round(np.mean([1 if len(v) >= 2 else 0 for v in team_assignments.values()]), 3)
    reward_score = round(np.mean(reward_vals), 3) if reward_vals else 0.0
    trust_weighted_coord = round(np.mean(trust_coord_vals), 3) if trust_coord_vals else 0.0

    return {
        "avg_trust": avg_trust,
        "trusturgency": round(avg_trust * avg_urgency, 3),
        "avg_spoofed": avg_spoofed,
        "avg_recovery": avg_recovery,
        "avg_distance": avg_distance,
        "avg_violations": avg_violations,
        "avg_duplicates": avg_duplicates,
        "avg_bait": bait_penalty,
        "avg_exit_violations": exit_violations,
        "avg_coord_success": avg_coord_success,
        "reward_score": reward_score,
        "trust_weighted_coord": trust_weighted_coord,
        "coalition_success": coalition_success
    }


In [ ]:
def run_benchmark_simulation(file_path, use_qubo=True):
    with open(file_path, 'r') as f:
        data = json.load(f)

    graph, tasks = build_benchmark_graph(data)
    robots = init_benchmark_robots(data)
    precedence_pairs = assign_chained_precedence(graph)
    exit_rooms = assign_exit_rooms(graph)
    dist_matrix = get_distance_matrix(data)

    sampler = neal.SimulatedAnnealingSampler()
    bqm = None

    trust_over_time = []
    trusturgency_over_time = []
    spoofed_over_time = []
    recovery_over_time = []
    coord_success_over_time = []
    violations_over_time = []
    distance_over_time = []
    bait_penalty_over_time = []
    exit_violations_over_time = []
    duplicates_over_time = []
    reward_score_over_time = []
    trust_weighted_over_time = []
    coalition_success_over_time = []

    initial_trust_avg = np.mean([r["trust"] for r in robots.values()])

    for round_num in tqdm(range(TOTAL_ROUNDS), desc=f"Running {'QUBO' if use_qubo else 'Greedy'}", leave=False):
        reset_spoofed_rooms(robots)
        update_dynamic_blockages(graph)
        decay_task_urgency(graph, decay_rate=0.2)
        apply_environmental_noise(robots)
        cure_spoofed_robots(robots)
        simulate_comms_failures(robots)

        precedence_map = define_precedence_map(precedence_pairs)

        if use_qubo:
            static_terms, quad_penalties = precompute_static_costs(robots, tasks, graph)
            cost_dict = update_dynamic_costs_np(
                robots, tasks, graph, static_terms, dist_matrix,
                dist_penalty=0.04, comp_penalty=1.0,
                local_radius=LOCAL_VIEW_RADIUS, duplicate_soft_penalty=5.0
            )
            bqm = build_or_update_bqm(bqm, cost_dict, quad_penalties)
            result = sampler.sample(bqm, num_reads=NUM_READS).first.sample
            assignments = apply_qubo_assignments(result, robots, tasks, precedence_map, graph)
        else:
            assignments = hardened_greedy_assignment(
                robots, tasks, graph, precedence_map, dist_matrix, local_radius=LOCAL_VIEW_RADIUS
            )

        for r_id, r in robots.items():
            if r_id not in assignments:
                r["idle_rounds"] = r.get("idle_rounds", 0) + 1
                decay = 0.005 * r["idle_rounds"]
                r["trust"] = max(0, r["trust"] - decay)
            else:
                r["idle_rounds"] = 0
            if r.get("pending_reward", 0) > 0.02:
                r["trust"] = min(1.0, r["trust"] + 0.005)

        spoofing_attack_target_high_trust(robots, tasks, dist_matrix, spoof_frac_range=SPOOF_FRAC_RANGE, severity=SPOOF_SEVERITY)

        update_urgency(graph, robots, tasks)

        metrics = run_metrics(robots, assignments, tasks, precedence_map, dist_matrix, graph, exit_rooms)

        trust_over_time.append(metrics["avg_trust"])
        trusturgency_over_time.append(metrics["trusturgency"])
        spoofed_over_time.append(metrics["avg_spoofed"])
        recovery_over_time.append(metrics["avg_recovery"])
        coord_success_over_time.append(metrics["avg_coord_success"])
        violations_over_time.append(metrics["avg_violations"])
        distance_over_time.append(metrics["avg_distance"])
        bait_penalty_over_time.append(metrics["avg_bait"])
        exit_violations_over_time.append(metrics["avg_exit_violations"])
        duplicates_over_time.append(metrics["avg_duplicates"])
        reward_score_over_time.append(metrics["reward_score"])
        trust_weighted_over_time.append(metrics["trust_weighted_coord"])
        coalition_success_over_time.append(metrics["coalition_success"])

    final_trust = np.mean(trust_over_time)
    delta_trust = final_trust - initial_trust_avg
    reward_score = np.mean(reward_score_over_time)
    trust_weighted_coord = np.mean(trust_weighted_over_time)

    trust_effort = trust_weighted_coord / (final_trust + 1e-6)
    delta_per_reward = delta_trust / (reward_score + 1e-6)

    return {
        "method": "QUBO" if use_qubo else "Greedy",
        "file": os.path.basename(file_path),
        "final_trust": final_trust,
        "initial_trust_avg": initial_trust_avg,
        "delta_trust": delta_trust,
        "delta_per_reward": delta_per_reward,
        "trust_effort": trust_effort,
        "trusturgency": np.mean(trusturgency_over_time),
        "avg_spoofed": np.mean(spoofed_over_time),
        "avg_recovery": np.mean(recovery_over_time),
        "avg_coord_success": np.mean(coord_success_over_time),
        "avg_violations": np.mean(violations_over_time),
        "avg_distance": np.mean(distance_over_time),
        "avg_bait": np.mean(bait_penalty_over_time),
        "avg_exit_violations": np.mean(exit_violations_over_time),
        "avg_duplicates": np.mean(duplicates_over_time),
        "reward_score": reward_score,
        "trust_weighted_coord": trust_weighted_coord,
        "coalition_success": np.mean(coalition_success_over_time)
    }



In [ ]:
all_summaries = []

for file_path in tqdm(benchmark_files, desc="Running QUBO Benchmarks"):
    try:
        result = run_benchmark_simulation(file_path, use_qubo=True)
        result["method"] = "QUBO"
        all_summaries.append(result)
    except json.JSONDecodeError:
        print(f"Skipping invalid or empty file during QUBO: {file_path}")
    except Exception as e:
        print(f"Unexpected error during QUBO run for {file_path}: {e}")

for file_path in tqdm(benchmark_files, desc="Running Greedy Benchmarks"):
    try:
        result = run_benchmark_simulation(file_path, use_qubo=False)
        result["method"] = "Greedy"
        all_summaries.append(result)
    except json.JSONDecodeError:
        print(f"Skipping invalid or empty file during Greedy: {file_path}")
    except Exception as e:
        print(f"Unexpected error during Greedy run for {file_path}: {e}")



Running QUBO Benchmarks:   0%|          | 0/106 [00:00<?, ?it/s]

Running QUBO:   0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Running QUBO:   0%|          | 0/100 [00:00<?, ?it/s]

Running QUBO:   0%|          | 0/100 [00:00<?, ?it/s]

...

Running Greedy:   0%|          | 0/100 [00:00<?, ?it/s]

Running Greedy:   0%|          | 0/100 [00:00<?, ?it/s]

Running Greedy:   0%|          | 0/100 [00:00<?, ?it/s]

Running Greedy:   0%|          | 0/100 [00:00<?, ?it/s]

Running Greedy:   0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
df_all = pd.DataFrame(all_summaries)
df_all.to_csv("benchmark_summary_all_raw_v4.csv", index=False)

df_qubo = df_all[df_all['method'] == "QUBO"]
df_greedy = df_all[df_all['method'] == "Greedy"]

df_qubo.mean(numeric_only=True).to_frame().T.to_csv("benchmark_summary_across_qubo_v4.csv", index=False)
df_greedy.mean(numeric_only=True).to_frame().T.to_csv("benchmark_summary_across_greedy_v4.csv", index=False)

print("All benchmark simulations complete and saved!")

All benchmark simulations complete and saved!
